In [138]:
import tushare as ts
import pandas as pd
#http://tushare.org/reference.html#id3

"""
https://xueqiu.com/2785562330/55607659
1、好行业
    环保、医疗保健、互联网+（信息安全）、新材料
2、好企业
3、好价格
    一个重要的买入观察及时指标就是(周）均线，尽可能在均线收拢是买入
"""
def prepare(year, quarter):
    # get data
    df_b0 = ts.get_stock_basics()
    df_prof0 = ts.get_profit_data(year,quarter)
    df_grow0 = ts.get_growth_data(year,quarter)
    df_deb0 = ts.get_debtpaying_data(year, quarter)
    # merge feature
    df_b0['mktcap'] = df_b0.apply(lambda row: row.pe * row.esp * row.totals, 
                                  axis=1)
    df_b = df_b0[['name', 'industry', 'pe', 'esp', 'mktcap', 'gpr','totals']]
    df_grow = df_grow0.set_index('code')[['nav', 'mbrg', 'nprg', 'epsg']]
    df_prof = df_prof0.set_index('code')[['roe', 'business_income', 'bips']]
    df_deb = df_deb0.set_index('code')[['sheqratio']]
    df = df_b.join(df_prof).join(df_grow).join(df_deb)
    df['peg'] = df.apply(lambda row: row.pe / max(1e-6, row.epsg), axis=1)
    df['minr'] = df.apply(lambda row: row.totals > 0 and row.bips/(
        row.business_income/100/row.totals) or 0, axis=1)
    df.drop(['bips','business_income'], axis = 1, inplace=True)
    return df

def feature(df, names):
    if isinstance(df['name'][0], unicode):
        names = [s.decode('utf8') for s in names]
    return df[df['name'].isin(names)]

def filter_kv(df, dic, more = True):
    for k,v in dic.iteritems():
        if more:
            df = df[df[k] >= v]
        else:
            df = df[df[k] <= v]
    return df

def good_stock(df):
    """
（1）市盈率增长率比 peg ＜1 
         peg= 市盈率（PE）/每股收益增长率(epsg)
（2）营业收入增长率 mbrg＞30% 
（3）净利润增长率 nprg ＞30%
（4）净资产收益率 roe ＞15%
（5）净资产增长率 nav > 8%
（6）毛利率 gpr＞50%
 (7）资产负债率 1 - sheqratio(股东权益率) ＜ 60%  
（8）市值尽可能小 mktcap
(9) 主营收入占比 > 80% minr
    """
    df = filter_kv(df, {'roe':15, 'gpr': 50, 'nprg':30, 
                        'mbrg': 30, 'nav': 8, 'sheqratio': 40, 'minr': 0.8}, True)
    df = filter_kv(df, {'mktcap':1e3, 'peg': 1}, False)
    return df

In [143]:
year, quarter = 2017,4
df = prepare(year, quarter)

[Getting data:]##########[Getting data:]#######[Getting data:]########

In [144]:
good_stock(df)

,name,industry,pe,esp,mktcap,gpr,totals,roe,nav,mbrg,nprg,epsg,sheqratio,peg,minr
code,,,,,,,,,,,,,,,
002677,浙江美大,家用电器,36.60,0.480,113.489280,53.94,6.46,23.32,13.6709,54.1812,50.5960,50.0000,78.0823,0.732000,0.999880
300146,汤臣倍健,食品,32.46,0.522,248.909123,67.08,14.69,15.02,9.1048,34.7182,51.0487,40.5405,83.9432,0.800681,0.999231
300740,御家汇,日用化工,53.29,0.991,84.496624,52.63,1.60,29.35,44.4849,40.6107,117.8902,112.9032,63.0505,0.471997,1.333324
600338,西藏珠峰,铅锌,20.63,1.705,229.687199,72.55,6.53,65.35,31.2005,67.0234,71.9849,71.2593,52.6689,0.289506,0.999967
600516,方大炭素,矿物制品,13.14,2.110,496.007406,76.59,17.89,37.31,68.4229,248.6204,13132.3664,5175.0000,73.0582,0.002539,1.000112


In [142]:
my_stocks = ('华友钴业', '兆易创新', '中南建设', '水井坊', '贵州茅台', '山西汾酒','片仔癀')
feature(df, my_stocks)
# epsg < 0 季度不靠谱
# 酒类靠谱
# g类，周期性，得环比去年

,name,industry,pe,esp,mktcap,gpr,totals,roe,nav,mbrg,nprg,epsg,sheqratio,peg,minr
code,,,,,,,,,,,,,,,
000961,中南建设,建筑施工,48.78,0.109,197.261442,17.87,37.10,2.91,-5.0284,-0.9725,9.1220,-23.0932,10.9991,4.878000e+07,1.000052
600436,片仔癀,中成药,56.46,1.118,380.627348,43.21,6.03,16.71,17.3580,68.8339,35.0511,40.0000,77.9313,1.411500e+00,0.999465
600519,贵州茅台,白酒,34.98,15.908,6989.160710,89.93,12.56,23.68,23.9850,59.3972,60.2488,60.3831,69.1664,5.793012e-01,0.999842
600779,水井坊,白酒,69.26,0.498,168.663337,78.88,4.89,16.35,6.7673,85.1825,63.2229,63.2045,60.1699,1.095808e+00,1.000910
600809,山西汾酒,白酒,46.39,0.930,373.615782,69.39,8.66,15.80,11.3296,42.7965,81.7039,78.5262,60.6077,5.907582e-01,1.000159
603799,华友钴业,小金属,49.69,1.852,545.713468,33.25,5.93,20.76,117.5545,76.1888,41327.9603,9150.0000,38.9175,5.430601e-03,1.000542
603986,兆易创新,半导体,86.68,1.700,299.132680,38.57,2.03,21.16,29.3085,44.6865,132.6599,-8.7473,75.1758,8.668000e+07,1.001505


In [43]:
df_fu = ts.forecast_data(2018,1)
feature(df_fu.set_index('code'),  my_stocks)

[Getting data:]####

,name,type,report_date,pre_eps,range
code,,,,,
300509,新美星,预升,2018-03-21,0.1500,-15.78%~1.07%
2709,天赐材料,预增,2018-03-21,0.2000,530%~560%
300457,赢合科技,预增,2018-03-21,0.3100,50%~65%
2449,国星光电,预增,2018-03-21,0.1088,30%~60%
300052,中青宝,预增,2018-03-21,0.0100,140%~170%
2404,嘉欣丝绸,预增,2018-03-21,0.0400,50%~80%
300436,广生堂,预降,2018-03-21,0.1438,-50%~-20%
300571,平治信息,预增,2018-03-20,0.5500,91.97%~119.4%
300446,乐凯新材,预升,2018-03-20,0.2484,-10%~10%


In [105]:
# 运营指标
df_y = ts.get_operation_data(2017,3)
feature(df_y,[s.decode('utf-8') for s in my_stocks])

[Getting data:]##########################################################

,code,name,arturnover,arturndays,inventory_turnover,inventory_days,currentasset_turnover,currentasset_days
2080,603986,兆易创新,13.9736,19.3222,2.0504,131.6816,0.9706,278.1784
2503,603799,华友钴业,8.0790,33.4200,1.3927,193.8680,0.8759,308.2544
2547,600436,片仔癀,6.7875,39.7790,1.3259,203.6353,0.7344,367.6471
2948,600809,山西汾酒,78.6900,3.4312,0.7623,354.1913,0.9086,297.1605
3178,600779,水井坊,56.0520,4.8170,0.3731,723.6666,0.8634,312.7172
3246,600519,贵州茅台,NaN,NaN,0.2060,1310.6796,0.4354,620.1194
3253,000961,中南建设,2.8553,94.5610,0.1832,1473.7991,0.1504,1795.2128
